# Ejecución Concurrente de Agentes (Consejo de Guerra)

## Objetivos de aprendizaje
- Ejecutar múltiples agentes **simultáneamente** (no secuencialmente)
- Usar `asyncio.gather()` para coordinación paralela
- Combinar perspectivas especializadas en tiempo real
- Medir mejoras de rendimiento vs ejecución secuencial
- Entender cuándo usar paralelo vs secuencial

## Idea clave
**Paralelo vs Secuencial:**

```
SECUENCIAL (lento):
Agente 1 [█████████] 5s
Agente 2          [█████████] 5s
Agente 3                    [█████████] 5s
Total: 15s

PARALELO (rápido):
Agente 1 [█████████]
Agente 2 [█████████] (simultáneamente)
Agente 3 [█████████]
Total: 5s
```

En el Consejo de Guerra: el Capitán necesita perspectivas de Técnica, Diplomacia y Seguridad **ahora**, no secuencialmente.

## Paso 0: Importaciones y configuración

In [1]:
import os
import time
from agent_framework import ChatAgent, ConcurrentBuilder, WorkflowOutputEvent
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv

load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("✅ Entorno cargado y Microsoft Agent Framework importado")

✅ Entorno cargado y Microsoft Agent Framework importado


## Paso 1: Crear tres agentes especializados
Cada uno con su expertise: técnica, diplomacia, seguridad.

In [2]:
# Agente 1: Oficial Técnico (Ingeniería)
agente_tech = ChatAgent(
    chat_client=OpenAIChatClient(
        base_url=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        model_id=AZURE_OPENAI_DEPLOYMENT
    ),
    name="OficialTécnico",
    instructions="""Eres el Jefe de Ingeniería de la Enterprise-D. Analizas aspectos técnicos y 
    de rendimiento de naves. Mantén respuestas concisas (2-3 oraciones). Sé práctico y enfocado 
    en viabilidad técnica."""
)

# Agente 2: Oficial Diplomático
agente_diplomacia = ChatAgent(
    chat_client=OpenAIChatClient(
        base_url=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        model_id=AZURE_OPENAI_DEPLOYMENT
    ),
    name="OficialDiplomático",
    instructions="""Eres el Oficial de Asuntos Diplomáticos. Analizas implicaciones políticas y 
    relaciones interplanetarias. Mantén respuestas concisas (2-3 oraciones). Considera múltiples 
    perspectivas (humanas, vulcanas, klingon, etc.)."""
)

# Agente 3: Jefe de Seguridad
agente_seguridad = ChatAgent(
    chat_client=OpenAIChatClient(
        base_url=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        model_id=AZURE_OPENAI_DEPLOYMENT
    ),
    name="OficialSeguridad",
    instructions="""Eres el Jefe de Seguridad. Analizas riesgos operacionales, amenazas Borg, 
    Romulan y contramedidas. Mantén respuestas concisas (2-3 oraciones). Sé exhaustivo identificando 
    peligros potenciales."""
)

print("✅ Tres agentes especialistas creados:")
print("  🔧 OficialTécnico")
print("  🤝 OficialDiplomático")
print("  ⚔️ OficialSeguridad")

✅ Tres agentes especialistas creados:
  🔧 OficialTécnico
  🤝 OficialDiplomático
  ⚔️ OficialSeguridad


In [3]:
# PATRÓN OFICIAL DE MICROSOFT AGENT FRAMEWORK
# ConcurrentBuilder crea un workflow donde:
# - Todos los agentes reciben: el MISMO prompt
# - Todos ejecutan: SIMULTÁNEAMENTE
# - Resultados: se agregan en lista de mensajes

workflow_paralelo = ConcurrentBuilder().participants([
    agente_tech,
    agente_diplomacia,
    agente_seguridad
]).build()

print("✅ Workflow paralelo creado con ConcurrentBuilder")

✅ Workflow paralelo creado con ConcurrentBuilder


## Paso 2: Crear workflow paralelo con ConcurrentBuilder (API oficial de Microsoft Agent Framework)
Con `ConcurrentBuilder`, el framework gestiona automáticamente:
- Ejecución paralela simultánea (todos los agentes reciben el mismo prompt)
- Agregación de resultados
- Optimización de rendimiento

## Paso 3: Ejecutar workflow paralelo
Los tres agentes corren simultáneamente, el framework agrega resultados automáticamente.

In [4]:
async def consejo_de_guerra():
    consulta = "¿Debemos enviar una expedición de contacto a la región Theta del Cuadrante Gamma?"
    
    print("\n" + "="*80)
    print("CONSEJO DE GUERRA - EJECUCIÓN PARALELA CON ConcurrentBuilder")
    print("="*80)
    print(f"\n📋 Pregunta: {consulta}\n")
    print("⏳ Ejecutando 3 agentes EN PARALELO (Microsoft Agent Framework)...\n")
    
    inicio_paralelo = time.time()
    
    # Ejecutar workflow y capturar eventos
    output_evt = None
    async for event in workflow_paralelo.run_stream(consulta):
        if isinstance(event, WorkflowOutputEvent):
            output_evt = event
            break
    
    tiempo_paralelo_total = time.time() - inicio_paralelo
    
    print(f"✅ Todos los análisis completados en {tiempo_paralelo_total:.2f}s (ejecución paralela)\n")
    
    if output_evt:
        messages = output_evt.data
        
        print("="*80)
        print("PERSPECTIVAS DE TODOS LOS AGENTES")
        print("="*80 + "\n")
        
        # Mostrar respuestas de todos los agentes
        for i, msg in enumerate(messages, 1): # type: ignore
            author = msg.author_name or ("usuario" if msg.role.value == "user" else "asistente")
            print(f"{i}. [{author}]:")
            print("-" * 80)
            content = msg.text if hasattr(msg, 'text') else str(msg.content)
            print(f"{content}\n")
        
        return {
            "consulta": consulta,
            "messages": messages,
            "tiempo_total": tiempo_paralelo_total
        }

# Ejecutar consejo de guerra
resultado = await consejo_de_guerra()


CONSEJO DE GUERRA - EJECUCIÓN PARALELA CON ConcurrentBuilder

📋 Pregunta: ¿Debemos enviar una expedición de contacto a la región Theta del Cuadrante Gamma?

⏳ Ejecutando 3 agentes EN PARALELO (Microsoft Agent Framework)...

✅ Todos los análisis completados en 2.48s (ejecución paralela)

PERSPECTIVAS DE TODOS LOS AGENTES

1. [usuario]:
--------------------------------------------------------------------------------
¿Debemos enviar una expedición de contacto a la región Theta del Cuadrante Gamma?

2. [OficialTécnico]:
--------------------------------------------------------------------------------
La región Theta del Cuadrante Gamma es altamente inexplorada y presenta riesgos técnicos significativos, incluyendo inestabilidad subespacial y falta de infraestructura de soporte. Recomiendo un despliegue inicial de sondas avanzadas para reconocimiento y evaluación de amenazas antes de considerar enviar una tripulación.

3. [OficialDiplomático]:
-----------------------------------------------

## Paso 4: Análisis de rendimiento
Ventajas de ConcurrentBuilder vs orquestación manual.

## Análisis de Rendimiento y Patrón

### 📊 Estadísticas de Ejecución

| Métrica | Valor |
|---------|-------|
| **Consulta** | ¿Debemos enviar una expedición de contacto a la región Theta? |
| **Agentes ejecutados** | 3 (en paralelo) |
| **Mensajes totales** | Múltiples (uno por agente) |
| **Tiempo total** | ~5-10 segundos (todos simultáneamente) |

### ✨ Ventajas de ConcurrentBuilder

| # | Ventaja | Descripción |
|---|---------|-------------|
| 1 | **API oficial** | Parte de Microsoft Agent Framework nativo |
| 2 | **Código limpio** | 4-5 líneas vs 40+ con `asyncio.gather()` |
| 3 | **Paralelo optimizado** | Framework maneja concurrencia automáticamente |
| 4 | **Streaming en tiempo real** | `WorkflowOutputEvent` proporciona eventos vivos |
| 5 | **Agregación automática** | Combina resultados de todos los agentes |
| 6 | **Documentación oficial** | Respaldado por Microsoft Learn |

### 🔧 Patrón Oficial

```python
workflow = ConcurrentBuilder().participants([agente1, agente2, agente3]).build()

async for event in workflow.run_stream(prompt):
    if isinstance(event, WorkflowOutputEvent):
        resultados = event.data
```

### 🎯 Cuándo Usar Paralelo vs Secuencial

#### PARALELO (ConcurrentBuilder)
✓ Múltiples agentes independientes
✓ Necesitas todas las perspectivas simultáneamente  
✓ El output de uno NO depende del otro
✓ Decisiones rápidas bajo presión

**Ejemplo:** Consejo de Guerra (Técnica + Diplomacia + Seguridad al mismo tiempo)

#### SECUENCIAL (SequentialBuilder)
✓ Un agente depende del output anterior
✓ Refinamiento progresivo (investigar → resumir)
✓ Transformación de datos entre etapas
✓ Pipelines de procesamiento

**Ejemplo:** Análisis profundo (Analista investiga → Redactor resume)

### ⏱️ Comparativa de Tiempo (Simulado)

```
SECUENCIAL (3 agentes, 5s cada uno):
Agente 1: [████████████████] 5s
Agente 2:                     [████████████████] 5s  
Agente 3:                                           [████████████████] 5s
TOTAL: 15 segundos

PARALELO (3 agentes, 5s cada uno):
Agente 1: [████████████████]
Agente 2: [████████████████] (simultáneamente)
Agente 3: [████████████████]
TOTAL: 5 segundos (~3x más rápido)
```

## Resumen de aprendizaje

**ConcurrentBuilder (Microsoft Agent Framework nativo):**
- Ejecuta múltiples agentes simultáneamente de forma optimizada
- Todos los agentes reciben el mismo prompt
- El framework agrega resultados automáticamente
- Código limpio: pasa de 40+ líneas con `asyncio.gather()` a 4-5 líneas

**En la Flota Estelar:**
- El Capitán Picard necesita la perspectiva de Ingeniería, Diplomacia y Seguridad AL MISMO TIEMPO
- En paralelo, obtiene todas las perspectivas más rápido
- Es la diferencia entre decisiones rápidas (paralelo) y decisiones lentas (secuencial)

**Comparativa:**
```python
# ANTES (manual con asyncio.gather()):
resultados = await asyncio.gather(
    agente_1.run(consulta),
    agente_2.run(consulta),
    agente_3.run(consulta)
)
# ... 40+ líneas de orquestación manual

# DESPUÉS (ConcurrentBuilder):
workflow = ConcurrentBuilder().participants([a1, a2, a3]).build()
async for event in workflow.run_stream(consulta):
    if isinstance(event, WorkflowOutputEvent):
        resultados = event.data  # ¡Hecho!
```

**Patrones disponibles en Microsoft Agent Framework:**
1. ✅ **ConcurrentBuilder** - Paralelo (acabas de usar)
2. ✅ **SequentialBuilder** - Secuencial (Tarea 13)
3. 🔜 **GroupChatBuilder** - Colaboración iterativa
4. 🔜 **MagenticBuilder** - Planificación compleja
5. 🔜 **HandoffBuilder** - Escalación dinámica